# Instagram Crawling - Jeju Hot Place

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [4]:
chromedriver = '/Users/nick/Downloads/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

In [5]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(4)

In [6]:
with open('password.txt') as f:
    password = f.read(9)

In [7]:
len(password)

9

In [8]:
email = 'lumenverum89@gmail.com'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [9]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(4)

## 문자인증은 수동으로 했음

In [10]:
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(4)
except:
    pass

In [11]:
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(4)
except:
    pass

## 제주도 맛집으로 검색

In [ ]:
from urllib.parse import quote
keyword = '제주도맛집'
search_url ='https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'

In [ ]:
driver.get(url)
time.sleep(8)

## 첫번째 게시글 열기

In [ ]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(4)

## 게시글 정보 가져오기

### 본문 내용 가져오기

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
import unicodedata
try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'.< 제주해물라면 맛집모음! > 일단저장@ 해물 아낌없이 주는 해물라면 맛집!바다뷰보며 먹는 해물라면은 말이 필요없음!저장해뒀다가 다음 여행시 참고하기!함께 하고픈 친구소환@..#제주맛집 #제주도맛집 #제주시맛집 #애월맛집 #중문맛집 #제주공항맛집 #서귀포맛집 #산방산맛집 #신제주맛집 #제주시청맛집 #제주애월맛집 #월정리맛집 #제주여행 #제주핫플'

### 본문 내용에서 해시태그 가져오기

In [ ]:
# '#[^\s#,\\]+'
import re
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#제주맛집',
 '#제주도맛집',
 '#제주시맛집',
 '#애월맛집',
 '#중문맛집',
 '#제주공항맛집',
 '#서귀포맛집',
 '#산방산맛집',
 '#신제주맛집',
 '#제주시청맛집',
 '#제주애월맛집',
 '#월정리맛집',
 '#제주여행',
 '#제주핫플']

### 작성일자 정보 가져오기

In [ ]:
date = soup.select_one('time.FH9sR.Nzb55')['datetime']
date

'2021-08-18T02:00:32.000Z'

In [ ]:
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-08-18'

### 좋아요 수 가져오기

In [ ]:
try:
    like = soup.select_one('div.Nm9Fw > a.zV_Nj').text[4:-1]
except:
    like = 0
like

'1,296'

### 위치 정보 가져오기

In [ ]:
try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ' '
place

''

In [ ]:
row = [content, date, like, place, tags]
row

['.< 제주해물라면 맛집모음! > 일단저장@ 해물 아낌없이 주는 해물라면 맛집!바다뷰보며 먹는 해물라면은 말이 필요없음!저장해뒀다가 다음 여행시 참고하기!함께 하고픈 친구소환@..#제주맛집 #제주도맛집 #제주시맛집 #애월맛집 #중문맛집 #제주공항맛집 #서귀포맛집 #산방산맛집 #신제주맛집 #제주시청맛집 #제주애월맛집 #월정리맛집 #제주여행 #제주핫플',
 '2021-08-18',
 '1,296',
 '',
 ['#제주맛집',
  '#제주도맛집',
  '#제주시맛집',
  '#애월맛집',
  '#중문맛집',
  '#제주공항맛집',
  '#서귀포맛집',
  '#산방산맛집',
  '#신제주맛집',
  '#제주시청맛집',
  '#제주애월맛집',
  '#월정리맛집',
  '#제주여행',
  '#제주핫플']]

### 다음 게시글로 이동

In [ ]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(4)

### 팝업 화면을 종료

In [ ]:
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

### 여러개의 게시글 가져오기

In [11]:
from tqdm.notebook import tqdm

In [ ]:
keyword = '제주도맛집'
search_url ='https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(8)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(4)

In [ ]:
results = []
for i in tqdm(range(100)):
    print(i, end=' ')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ' '
    row = [content, date, like, place, tags]
    results.append(row)
    driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
    time.sleep(4)

  0%|          | 0/100 [00:00<?, ?it/s]

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

In [ ]:
df = pd.DataFrame(results, columns=['content', 'date', 'like', 'place', 'tags'])
df.head()

,content,date,like,place,tags
0,(협찬)애월바다와 선셋🌅감성까지 더 한스페셜 카레맛집!😋다양한 카레와 토핑들로특별함...,2021-08-18,0,,"[#사이카레@saicurry_aewolOPEN, #사이카레, #구엄리맛집, #애월맛..."
1,"협찬제주애월맛집 ""랍스터빈""주소 : 제주시 애월읍 애월해안로 752전화 : 064-...",2021-08-18,0,,"[#애월맛집, #애월카페, #애월읍맛집, #제주애월맛집, #제주도맛집, #제주맛집,..."
2,"협찬제주썬셋맛집 ""랍스터빈""주소 : 제주시 애월읍 애월해안로 752전화 : 064-...",2021-08-18,0,,"[#애월맛집, #애월카페, #애월읍맛집, #제주애월맛집, #제주도맛집, #제주맛집,..."
3,<광고>노을 지는거 보면서 식사와 술한잔 하기에너무 좋은 #코시롱✔제주 제주시 애월...,2021-08-18,0,,"[#코시롱✔제주, #애월맛집, #애월읍맛집, #애월맛집추천, #애월흑돼지맛집, #제..."
4,"협찬제주애월 썬셋맛집 ""랍스터빈""주소 : 제주시 애월읍 애월해안로 752전화 : 0...",2021-08-18,0,,"[#애월맛집, #애월카페, #애월읍맛집, #제주애월맛집, #제주도맛집, #제주맛집,..."


## 제주도맛집, 제주맛집, 제주도관광, 제주여행

In [12]:
from urllib.parse import quote
from tqdm.notebook import tqdm

In [13]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    return [content, tags, date, like, place]

In [ ]:
search_url ='https://www.instagram.com/explore/tags/'
for keyword in ['제주도맛집', '제주맛집', '제주도관광', '제주여행']:
    url = f'{search_url}{quote(keyword)}'
    driver.get(url)
    time.sleep(8)
    driver.find_element_by_css_selector('div._9AhH0').click()
    time.sleep(4)
    results = []
    for _ in tqdm(range(20)):
        try:
            row = get_content(driver)
            results.append(row)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(4)
        except:
            time.sleep(8)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(4)
    df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
    df.to_csv(f'data/{keyword}.csv', index=False)
    driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
    time.sleep(4)

In [ ]:
driver.close()

### 중복을 제거하고 통합하여 저장하기

In [ ]:
for keyword in ['제주도맛집', '제주맛집', '제주도관광', '제주여행']:
    df = pd.read_csv(f'data/{keyword}.csv')
    print(df.shape)

In [ ]:
jeju_df = pd.DataFrame([])
for keyword in ['제주도맛집', '제주맛집', '제주도관광', '제주여행']:
    df = pd.read_csv(f'data/{keyword}.csv')

In [ ]:
jeju_df.tail()